In [ ]:
%pip install --quiet git+https://github.com/mozilla-ai/osm-ai-helper.git
%pip install --quiet ultralytics

In [ ]:
from osm_ai_helper.download_osm import download_osm

In [ ]:
TRAIN_AREA = "Ponteareas"
TEST_AREA = "Mondariz"

In [ ]:
download_osm(
    area=TRAIN_AREA,
    output_dir="dataset",
    selector="leisure=swimming_pool",
    discard={"location": "indoor"},
)

In [ ]:
download_osm(
    area=TEST_AREA,
    output_dir="dataset",
    selector="leisure=swimming_pool",
    discard={"location": "indoor"},
)

In [ ]:
from google.colab import userdata

from osm_ai_helper.group_elements_and_download_tiles import (
    group_elements_and_download_tiles,
)

In [ ]:
group_elements_and_download_tiles(
    f"dataset/{TRAIN_AREA}.json",
    f"dataset/{TRAIN_AREA}",
    userdata.get("MAPBOX_TOKEN")
)

In [ ]:
group_elements_and_download_tiles(
    f"dataset/{TEST_AREA}.json",
    f"dataset/{TEST_AREA}",
    userdata.get("MAPBOX_TOKEN")
)

In [ ]:
from osm_ai_helper.convert_to_yolo_dataset import convert_to_yolo_dataset

In [ ]:
convert_to_yolo_dataset(f"dataset/{TRAIN_AREA}")

In [ ]:
convert_to_yolo_dataset(f"dataset/{TEST_AREA}")

In [ ]:
from pathlib import Path

Path("yolo_dataset.yaml").write_text(
f"""
path: ./dataset
train: {TRAIN_AREA}
val: {TEST_AREA}

names:
  0: swimming_pool
"""
)

# Finetune model

In [ ]:
from ultralytics import YOLO

In [ ]:
yolo = YOLO("yolo11m.pt")

In [ ]:
yolo.train(data="yolo_dataset.yaml", epochs=1, imgsz=512)